In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from Fixed_Income_Toolbox import *

In [2]:
maturity = np.arange(1,7)
spot = [0.05, 0.055, 0.057, 0.059, 0.06, 0.061]
price = []
for i in range(len(maturity)):
    bond = ZeroCouponBond(100, maturity[i])
    price.append(bond.get_price(spot[i]))
price

[95.23809523809524,
 89.84524157139327,
 84.6788669093383,
 79.50897588580243,
 74.7258172866057,
 70.09833403416522]

In [3]:
def ho_and_lee_calibrate(level, spot, vol, price, fv=100):
    calibrate_lv = [s + vol for s in spot[level - 1]] + [spot[level - 1][-1] - vol] # construct the last level of spot
    def function(x, *args):
        calibrate_lv, spot, price, fv = args
        price_tree = [fv / (1 + spot + x) for spot in calibrate_lv]
        for i in reversed(range(len(price_tree))):
            for j in range(i):
                price_tree[j] = (0.5 * price_tree[j] + 0.5 * price_tree[j+1]) / (1 + spot[i][j])
        return price_tree[0] - price
    args = (calibrate_lv, spot, price, fv)
    m = fsolve(function, 0.01, args=args)
    return [s + m[0] for s in calibrate_lv]

In [4]:
# test calibration using the example in class
spot_dict = {1:[0.05263], 2:[0.07593, 0.03593]}
ho_and_lee_calibrate(3, spot_dict, 0.02, 85)

[0.09996345790671082, 0.05996345790671081, 0.01996345790671081]

In [5]:
# construct the ho and lee tree
vol = 0.015
spot_dict = {1:[spot[0]]}
for lv in range(2, len(price)+1):
    spot_dict[lv] = ho_and_lee_calibrate(lv, spot_dict, vol, price[lv-1])
spot_dict

{1: [0.05],
 2: [0.07523602642117286, 0.04523602642117285],
 3: [0.09164759479639102, 0.061647594796391016, 0.03164759479639102],
 4: [0.11129248303472124,
  0.08129248303472124,
  0.05129248303472124,
  0.021292483034721237],
 5: [0.1261246338504627,
  0.09612463385046267,
  0.06612463385046267,
  0.03612463385046268,
  0.006124633850462678],
 6: [0.14418404774940974,
  0.11418404774940971,
  0.08418404774940971,
  0.05418404774940973,
  0.024184047749409722,
  -0.005815952250590278]}

In [6]:
def get_payoff_tree(spot_dict, cf_dict):
    sorted_lv = sorted(spot_dict.keys(), reverse=True)
    last_period = sorted_lv[0]
    payoff_dict = {}
    for lv in sorted_lv:
        if lv == last_period:
            payoff_dict[lv] = np.divide(cf_dict[lv], [s + 1 for s in spot_dict[lv]]).tolist()
        else:
            payoff_dict[lv] = np.zeros(lv).tolist()
            for i in range(lv):
                payoff_dict[lv][i] = (0.5 * (payoff_dict[lv+1][i] + payoff_dict[lv+1][i+1]) + cf_dict[lv][i]) / (1 + spot_dict[lv][i])
    return payoff_dict

In [7]:
# part a
bond = ZeroCouponBond(100, 6)
z_ls = [bond.get_discount_function(0.055, t) for t in range(1,7)]
fixed_pay = 100 / sum(z_ls)
pv_ls = [z * fixed_pay for z in z_ls]
duration_a = np.dot(pv_ls, np.arange(1,7)) / sum(pv_ls)
duration_a

3.3441148048312574

In [9]:
# part b
# first get payoff at each node
fixed_cf = {i:[fixed_pay]*i for i in range(1, 7)}
fixed_payoff = get_payoff_tree(spot_dict, fixed_cf)
# then construct a new cash flow tree that computes extra payoff if 5.5% interest rate is quoted
cf_dict = {i:[0]*i for i in range(1, 7)}
for key in spot_dict:
    for i in range(key):
        cf_dict[key][i] = fixed_pay * (spot_dict[key][i] - 0.055)
# get rid of the nodes that have negative cashflow
# then add one time payment from the initial payoff tree
option_payoff = get_payoff_tree(spot_dict, cf_dict)
option_cf = {i:[0]*i for i in range(1, 7)}
threshold = 6
for lv in range(1, 7):
    for i in range(lv):
        if option_payoff[lv][i] >=0 and i < threshold:
            option_cf[lv][i] = fixed_pay
        elif option_payoff[lv][i] < 0 and i < threshold:
            option_cf[lv][i] = fixed_payoff[lv][i]
            threshold = i
# using the new cash flow tree, get the payoff again
final_payoff = get_payoff_tree(spot_dict, option_cf)
print(final_payoff)
duration_b = (final_payoff[2][1] - final_payoff[2][0]) / ((spot_dict[2][1] - spot_dict[2][0]) * final_payoff[1][0])
duration_b

{6: [17.495345090340024, 0.0, 0.0, 0.0, 0.0, 0.0], 5: [25.543857618185967, 0.0, 0.0, 0.0, 0.0], 4: [29.506024806168433, 0.0, 0.0, 0.0], 3: [31.851769134165988, 0.0, 0.0], 2: [33.428734199835795, 83.46154377869296], 1: [74.72669881095466]}


-22.31813188368427